In [14]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = '../data/cases_summary.csv'  # Replace with the actual path

# Read only the first 5 rows of the CSV file into a DataFrame
df = pd.read_csv(csv_file_path, nrows=215)

# Display the DataFrame
print(df)

# Display the first few rows
df.head()  # By default, this shows the first 5 rows


          id                                               data  \
0    1036848  {"id": 1036848, "name": "Roy E. Rice et al., R...   
1     522235  {"id": 522235, "name": "Donald S. Myhr & anoth...   
2     849980  {"id": 849980, "name": "Felix L. Litka et al.,...   
3    8504476  {"id": 8504476, "name": "A. DeLambre vs. M. J....   
4     999390  {"id": 999390, "name": "George Elves et al., R...   
..       ...                                                ...   
210   436063  {"id": 436063, "name": "Queen City Farms, Inc....   
211   488995  {"id": 488995, "name": "David H. Fulton & anot...   
212   508747  {"id": 508747, "name": "Cole Drug Company of M...   
213   674990  {"id": 674990, "name": "Woolworth Company, App...   
214   782360  {"id": 782360, "name": "Martin C. Peterson et ...   

                                    description_vector  \
0    [0.018131,0.05310722,-0.017784273,0.027015913,...   
1    [0.03384037,0.047954552,0.04315629,0.056288373...   
2    [0.04118457,0.04

,id,data,description_vector,summary,summary_vector
0,1036848,"{""id"": 1036848, ""name"": ""Roy E. Rice et al., R...","[0.018131,0.05310722,-0.017784273,0.027015913,...","In this case, Mr. and Mrs. Rice sued Valley Pl...","[0.05087421,0.03367086,0.002191651,0.029408086..."
1,522235,"{""id"": 522235, ""name"": ""Donald S. Myhr & anoth...","[0.03384037,0.047954552,0.04315629,0.056288373...",The court affirmed a decree regarding a disput...,"[0.061744608,0.053280853,0.058430485,0.0306428..."
2,849980,"{""id"": 849980, ""name"": ""Felix L. Litka et al.,...","[0.04118457,0.04003745,0.013101003,0.058699206...","In this case, the respondents owned property w...","[0.036264345,0.010527464,0.04393831,0.05477051..."
3,8504476,"{""id"": 8504476, ""name"": ""A. DeLambre vs. M. J....","[0.01810683,0.048254702,-0.010637762,0.0007591...",The court delivered an opinion on a suit for d...,"[0.023722136,0.021588398,-0.000580502,-0.00791..."
4,999390,"{""id"": 999390, ""name"": ""George Elves et al., R...","[0.05829014,0.019327164,0.02624967,0.07395764,...",The town of Houghton appealed a judgment that ...,"[0.05732037,0.005931553,0.021043532,0.05667329..."


In [11]:
import json
import subprocess
from sqlalchemy.ext.asyncio import AsyncEngine, create_async_engine
from sqlalchemy import event
import logging

logger = logging.getLogger("legalcaseapp")
from azure.identity import DefaultAzureCredential

def get_password_from_azure_credential():
    """Retrieve an Azure AD access token using DefaultAzureCredential."""
    credential = DefaultAzureCredential()
    token = credential.get_token("https://ossrdbms-aad.database.windows.net/.default")
    return token.token


async def create_postgres_engine(*, host, username, database, sslmode, azure_credential=None) -> AsyncEngine:

    token_based_password = False
    # Check if host is an Azure Database for PostgreSQL instance
    if host.endswith(".database.azure.com"):
        token_based_password = True
        logger.info("Authenticating to Azure Database for PostgreSQL using Azure Identity...")
        password = get_password_from_azure_credential()  # Get token as password
    else:
        raise ValueError("Azure credential must be provided for Azure Database for PostgreSQL")

    # Construct the DATABASE_URI
    DATABASE_URI = f"postgresql+asyncpg://{username}:{password}@{host}/{database}"
    if sslmode:
        DATABASE_URI += f"?sslmode={sslmode}"

    engine = create_async_engine(DATABASE_URI, echo=False)

    # Update the token if required before each connection
    @event.listens_for(engine.sync_engine, "do_connect")
    def update_password_token(dialect, conn_rec, cargs, cparams):
        if token_based_password:
            logger.info("Refreshing access token for Azure Database for PostgreSQL")
            cparams["password"] = get_password_from_azure_credential()

    return engine


In [12]:
from azure.identity import DefaultAzureCredential
from sqlalchemy.ext.asyncio import AsyncEngine

# Use DefaultAzureCredential to automatically find the right credentials
azure_credential = DefaultAzureCredential()

# Define your connection parameters
host = "legalcase-demo-3cxtrqd7sbayk-postgresql.postgres.database.azure.com"
username = "legalcase-demo-3cxtrqd7sbayk-id-web"
database = "postgres"
sslmode = "require"

# Assuming create_postgres_engine is the function you provided
engine = await create_postgres_engine(
    host=host,
    username=username,
    database=database,
    sslmode=sslmode
)

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Please run 'az login' to set up an account
	AzurePowerShellCredential: PowerShell is not installed
	AzureDeveloperCliCredential: Please run 'azd auth login' from a command prompt to authenticate before using this credential.
To mitigate this issue, please refer to the troubleshooting guidelines here at https://aka.ms/azsdk/python/identity/defaultazurecredential/troubleshoot.


ClientAuthenticationError: DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Please run 'az login' to set up an account
	AzurePowerShellCredential: PowerShell is not installed
	AzureDeveloperCliCredential: Please run 'azd auth login' from a command prompt to authenticate before using this credential.
To mitigate this issue, please refer to the troubleshooting guidelines here at https://aka.ms/azsdk/python/identity/defaultazurecredential/troubleshoot.

In [61]:
import logging
import os
import sys

import psycopg2
from dotenv import load_dotenv
from psycopg2.extras import Json

# Load environment variables
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("graph_db_ingestion")

# Define database connection parameters
conn_params = {
    "dbname": os.getenv("POSTGRES_DATABASE", "postgres"),
    "user": os.getenv("POSTGRES_USERNAME", "postgres"),
    "password": os.getenv("POSTGRES_PASSWORD", "Passwd34!"),
    "host": os.getenv("POSTGRES_HOST", "localhost"),
    "port": os.getenv("POSTGRES_PORT", "5432"),
}

In [62]:
def drop_age_graph(conn, graph_name):
    with conn.cursor() as cur:
        # Drop the graph using ag_catalog.drop_graph with cascade
        cur.execute(f"SELECT * FROM ag_catalog.drop_graph('{graph_name}', true);")
        conn.commit()
        logger.info(f"Graph '{graph_name}' dropped successfully.")

In [59]:


def create_edges_from_citations(conn):
    """
    Creates edges in the `case_graph` graph based on citation relationships
    in the `cases_summary` table.
    """
    with conn.cursor() as cur:
        try:
            cur.execute("LOAD 'age';")
            cur.execute("SET search_path = ag_catalog, public;")

            # Step 1: Define edges based on citations in cases_summary
            define_edges_query = """
            WITH edges AS (
                SELECT DISTINCT c1.id AS id_from, c2.id AS id_to
                FROM cases_summary c1
                LEFT JOIN LATERAL jsonb_array_elements(c1.data -> 'cites_to') AS cites_to_element ON true
                LEFT JOIN LATERAL jsonb_array_elements(cites_to_element -> 'case_ids') AS case_ids ON true
                JOIN cases_summary c2 ON case_ids::text = c2.id
            )
            SELECT id_from, id_to FROM edges;
            """

            # Step 2: Execute the query to fetch edge pairs
            cur.execute(define_edges_query)
            edges = cur.fetchall()
            
            # Step 3: Insert edges into the graph one by one
            for id_from, id_to in edges:
                cypher_query = f"""
                SELECT * FROM ag_catalog.cypher('case_graph', $$
                    MATCH (a:Case), (b:Case)
                    WHERE a.id = '{id_from}' AND b.id = '{id_to}'
                    CREATE (a)-[:CITES]->(b)
                    RETURN a, b
                $$) AS (a ag_catalog.agtype, b ag_catalog.agtype);
                """
                cur.execute(cypher_query)
            
            # Commit after inserting edges
            conn.commit()
            logger.info("Edges created successfully in the `case_graph` based on citation relationships.")

        except psycopg2.Error as e:
            # Roll back if there is an error and log it
            conn.rollback()
            logger.error(f"Failed to create edges: {e}")



In [60]:
with psycopg2.connect(**conn_params) as conn:
    # Step 1: Setup the AGE graph
    create_edges_from_citations(conn)


INFO:graph_db_ingestion:Edges created successfully in the `case_graph` based on citation relationships.


In [76]:
def drop_graph(conn):
    graph_name = "case_graph"
    with conn.cursor() as cur:
        # Check if the graph already exists
        cur.execute(f"SELECT * FROM ag_catalog.ag_graph WHERE name = '{graph_name}';")
        graph_exists = cur.fetchone()

        if graph_exists:
            # If the graph exists, drop it
            logger.info(f"Graph '{graph_name}' already exists. Dropping it.")
            drop_age_graph(conn, graph_name)

with psycopg2.connect(**conn_params) as conn:
    # Step 1: Setup the AGE graph
    drop_graph(conn)


INFO:graph_db_ingestion:Graph 'case_graph' already exists. Dropping it.
INFO:graph_db_ingestion:Graph 'case_graph' dropped successfully.


In [2]:
def setup_age_graph(conn):
    graph_name = "case_graph"
    with conn.cursor() as cur:
        # Ensure the AGE extension is enabled and loaded
        cur.execute("CREATE EXTENSION IF NOT EXISTS age;")
        cur.execute("LOAD 'age';")

        # Set the search_path to include the Apache AGE catalog
        cur.execute('SET search_path = ag_catalog, "$user", public;')

        conn.commit()

        # Create the new graph
        cur.execute(f"SELECT create_graph('{graph_name}');")

        conn.commit()

        logger.info(f"Graph '{graph_name}' created successfully in Apache AGE.")

In [3]:
with psycopg2.connect(**conn_params) as conn:
    # Step 1: Setup the AGE graph
    setup_age_graph(conn)


INFO:graph_db_ingestion:Graph 'case_graph' created successfully in Apache AGE.


In [1]:
import pandas as pd
import json
import openai
import numpy as np
import psycopg2
from psycopg2.extras import Json
import os
from dotenv import load_dotenv
import time

# Load environment variables
load_dotenv()

# Database setup
conn_params = {
    "dbname": os.getenv("POSTGRES_DATABASE"),
    "user": os.getenv("POSTGRES_USERNAME"),
    "password": os.getenv("POSTGRES_PASSWORD"),
    "host": os.getenv("POSTGRES_HOST"),
    "port": os.getenv("POSTGRES_PORT"),
}

# OpenAI setup for Azure
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_version = os.getenv("AZURE_OPENAI_VERSION")
deployment_id = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")

print("OpenAI endpoint:", openai.api_base)
print("OpenAI version:", openai.api_version)
print("OpenAI deployment ID:", deployment_id)


def generate_summary_with_retry(text, deployment=None, max_retries=5):
    if deployment is None:
        deployment = deployment_id
    for attempt in range(max_retries):
        try:
            response = openai.ChatCompletion.create(
                engine=deployment,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant trained to summarize legal texts."},
                    {"role": "user", "content": f"Summarize the following legal text:\n\n{text}"}
                ],
                max_tokens=150,
                temperature=0.3,
            )
            # If successful, return the summary
            summary = response.choices[0].message['content'].strip()
            return summary
        except openai.error.RateLimitError as e:
            # Check for 'Retry-After' in response headers
            retry_after = int(e.headers.get("Retry-After", 10))  # Default to 10 seconds if not provided
            print(f"Rate limit hit. Retrying in {retry_after} seconds...")
            time.sleep(retry_after)
        except Exception as e:
            print(f"Error generating summary: {e}")
            break  # Break on other types of errors not related to rate limits

    print("Failed to generate summary after multiple retries.")
    return None


# Connect to the database
with psycopg2.connect(**conn_params) as conn:
    with conn.cursor() as cursor:
        # Add summary column if not exists
        cursor.execute("""
            ALTER TABLE demo_cases_updated
            ADD COLUMN IF NOT EXISTS summary TEXT;
        """)
        conn.commit()
        
        # Query demo_cases for entries needing a summary
        select_query = """
            SELECT id, data
            FROM demo_cases_updated
            WHERE summary IS NULL;
        """
        cursor.execute(select_query)
        cases = cursor.fetchall()
        
        # Process each case
        for case_id, data_dict in cases:
            data = data_dict  # Directly use `data_dict` as it is already a dictionary
            opinions = data.get('casebody', {}).get('opinions', [])
            
            if opinions:
                # Concatenate all opinion texts
                text_to_summarize = " ".join(opinion['text'] for opinion in opinions if 'text' in opinion)
                
                # Generate summary
                summary = generate_summary_with_retry(text_to_summarize)
                
                # Update table with summary
                if summary:
                    update_query = """
                        UPDATE demo_cases_updated
                        SET summary = %s
                        WHERE id = %s;
                    """
                    cursor.execute(update_query, (summary, case_id))
                    conn.commit()
                    print(f"Summary for case {case_id} added.")
                else:
                    print(f"Failed to generate summary for case {case_id}.")
            else:
                print(f"No opinion text found for case {case_id}.")


OpenAI endpoint: https://serdar-azure-open-ai.openai.azure.com
OpenAI version: 2024-08-01-preview
OpenAI deployment ID: gpt-4o
Summary for case 1017660 added.
Rate limit hit. Retrying in 58 seconds...
Summary for case 1043159 added.
Summary for case 1346648 added.
Rate limit hit. Retrying in 56 seconds...
Summary for case 3320973 added.
Summary for case 3322332 added.
Summary for case 3329625 added.
Summary for case 3335022 added.
Rate limit hit. Retrying in 52 seconds...
Summary for case 3335075 added.
Summary for case 3335809 added.
Summary for case 3841816 added.
Summary for case 3852931 added.
Summary for case 3862445 added.
Rate limit hit. Retrying in 50 seconds...
Summary for case 4273930 added.
Rate limit hit. Retrying in 1 seconds...
Summary for case 473788 added.
Rate limit hit. Retrying in 55 seconds...
Summary for case 4938756 added.
Summary for case 498084 added.
Summary for case 5041745 added.
Rate limit hit. Retrying in 54 seconds...
Summary for case 520681 added.
Summary

KeyboardInterrupt: 

In [13]:
import pandas as pd
import json
import openai
import numpy as np
import psycopg2
from psycopg2.extensions import register_adapter, AsIs
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Database setup
def addapt_vector(nparray):
    """Adapt a numpy array to the PostgreSQL VECTOR type."""
    vector_str = ','.join(map(str, nparray))
    return AsIs(f"'[{vector_str}]'::VECTOR")

register_adapter(np.ndarray, addapt_vector)

conn_params = {
    "dbname": os.getenv("POSTGRES_DATABASE"),
    "user": os.getenv("POSTGRES_USERNAME"),
    "password": os.getenv("POSTGRES_PASSWORD"),
    "host": os.getenv("POSTGRES_HOST"),
    "port": os.getenv("POSTGRES_PORT"),
}

# OpenAI setup for Azure
embed_deployment_id = os.getenv("AZURE_OPENAI_EMBED_DEPLOYMENT")
embed_model = os.getenv("AZURE_OPENAI_EMBED_MODEL")
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_version = '2023-05-15'

# Function to generate embeddings using Azure OpenAI
def create_embeddings(text, deployment=None):
    if deployment is None:
        deployment = embed_deployment_id
    try:
        response = openai.Embedding.create(input=text, deployment_id=deployment)
        embedding = response['data'][0]['embedding']
        return np.array(embedding)
    except Exception as e:
        print(f"Error generating embeddings: {e}")
        return None

# Connect to the database and add embeddings to `summary_vector`
with psycopg2.connect(**conn_params) as conn:
    with conn.cursor() as cursor:
        # Add summary_vector column if it doesn't exist
        cursor.execute("""
            ALTER TABLE demo_cases
            ADD COLUMN IF NOT EXISTS summary_vector VECTOR(1536);
        """)
        conn.commit()

        # Query demo_cases for entries needing summary_vector embeddings
        select_query = """
            SELECT id, summary
            FROM demo_cases
            WHERE summary IS NOT NULL AND summary_vector IS NULL;
        """
        cursor.execute(select_query)
        cases = cursor.fetchall()
        
        # Process each case to generate embeddings
        for case_id, summary_text in cases:
            # Generate embedding for the summary
            embedding_vector = create_embeddings(summary_text)
            
            # Update table with the embedding vector if generation was successful
            if embedding_vector is not None:
                update_query = """
                    UPDATE demo_cases
                    SET summary_vector = %s
                    WHERE id = %s;
                """
                cursor.execute(update_query, (embedding_vector, case_id))
                conn.commit()
                print(f"Embedding for case {case_id} added to summary_vector.")
            else:
                print(f"Failed to generate embedding for case {case_id}.")


Embedding for case 819052 added to summary_vector.
Embedding for case 1036848 added to summary_vector.
Embedding for case 522235 added to summary_vector.
Embedding for case 849980 added to summary_vector.
Embedding for case 8504476 added to summary_vector.
Embedding for case 98993 added to summary_vector.
Embedding for case 999390 added to summary_vector.
Embedding for case 3335075 added to summary_vector.
Embedding for case 3335809 added to summary_vector.
Embedding for case 3329625 added to summary_vector.
Embedding for case 3335022 added to summary_vector.
Embedding for case 512229 added to summary_vector.
Embedding for case 881896 added to summary_vector.
Embedding for case 1034630 added to summary_vector.
Embedding for case 1091298 added to summary_vector.
Embedding for case 1151217 added to summary_vector.
Embedding for case 1201345 added to summary_vector.
Embedding for case 1791675 added to summary_vector.
Embedding for case 918583 added to summary_vector.
Embedding for case 93

In [79]:
import networkx as nx
from graspologic.partition import hierarchical_leiden

G = nx.Graph()

edges = [
    (1, 2, 0.8),
    (2, 3, 0.6),
    (3, 4, 0.9),
    (4, 5, 0.4),
    (5, 1, 0.7),
]

for edge in edges:
    G.add_edge(edge[0], edge[1], weight=edge[2])


max_cluster_size = 2
resolution = 1.0


clusters = hierarchical_leiden(
    graph=G,
    max_cluster_size=max_cluster_size,
    resolution=resolution,
    is_weighted=True,
    weight_attribute="weight"
)

for node, community in clusters.final_level_hierarchical_clustering().items():
    print(f"Node {node} is in community {community}")


Node 1 is in community 0
Node 2 is in community 0
Node 5 is in community 0
Node 3 is in community 1
Node 4 is in community 1


In [1]:
import pandas as pd
import json
import openai
import numpy as np
import psycopg2
from psycopg2.extensions import register_adapter, AsIs
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Database setup
def addapt_vector(nparray):
    """Adapt a numpy array to the PostgreSQL VECTOR type."""
    vector_str = ','.join(map(str, nparray))
    return AsIs(f"'[{vector_str}]'::VECTOR")

register_adapter(np.ndarray, addapt_vector)

conn_params = {
    "dbname": os.getenv("POSTGRES_DATABASE"),
    "user": os.getenv("POSTGRES_USERNAME"),
    "password": os.getenv("POSTGRES_PASSWORD"),
    "host": os.getenv("POSTGRES_HOST"),
    "port": os.getenv("POSTGRES_PORT"),
}

# OpenAI setup for Azure
embed_deployment_id = os.getenv("AZURE_OPENAI_EMBED_DEPLOYMENT")
embed_model = os.getenv("AZURE_OPENAI_EMBED_MODEL")
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_version = '2023-05-15'

# Function to generate embeddings using Azure OpenAI
def create_embeddings(text, deployment=None):
    if deployment is None:
        deployment = embed_deployment_id
    try:
        response = openai.Embedding.create(input=text, deployment_id=deployment)
        embedding = response['data'][0]['embedding']
        return np.array(embedding)
    except Exception as e:
        print(f"Error generating embeddings: {e}")
        return None

# Connect to the database and add embeddings to `full_content_vector`
with psycopg2.connect(**conn_params) as conn:
    with conn.cursor() as cursor:
        # Add full_content_vector column if it doesn't exist
        cursor.execute("""
            ALTER TABLE final_community_reports
            ADD COLUMN IF NOT EXISTS full_content_vector VECTOR(1536);
        """)
        conn.commit()

        # Query final_community_reports for entries needing full_content_vector embeddings
        select_query = """
            SELECT id, full_content
            FROM final_community_reports
            WHERE full_content IS NOT NULL AND full_content_vector IS NULL;
        """
        cursor.execute(select_query)
        reports = cursor.fetchall()
        
        # Process each report to generate embeddings
        for report_id, full_content_text in reports:
            # Generate embedding for the full_content
            embedding_vector = create_embeddings(full_content_text)
            
            # Update table with the embedding vector if generation was successful
            if embedding_vector is not None:
                update_query = """
                    UPDATE final_community_reports
                    SET full_content_vector = %s
                    WHERE id = %s;
                """
                cursor.execute(update_query, (embedding_vector, report_id))
                conn.commit()
                print(f"Embedding for report {report_id} added to full_content_vector.")
            else:
                print(f"Failed to generate embedding for report {report_id}.")


Embedding for report e8d07459-ae88-41ae-a7e2-6abb92fdb7a9 added to full_content_vector.
Embedding for report c14820fe-8dfc-49ed-8a9a-5623d8f4495b added to full_content_vector.
Embedding for report be3cea6b-7458-4dc6-998d-c0b8544edc99 added to full_content_vector.
Embedding for report f7837287-bd2f-444d-bd8b-cf4b72fd688c added to full_content_vector.
Embedding for report 3ed6c987-5093-4be8-90d9-5b4d48950af0 added to full_content_vector.
Embedding for report 6d086ee9-55a4-48a3-a812-91826a51d541 added to full_content_vector.
Embedding for report e4c823e3-1545-4337-8372-24f6b3006a72 added to full_content_vector.
Embedding for report b3beafc4-fc9f-4205-96e6-f6390c47ac33 added to full_content_vector.
Embedding for report acc1ba95-e582-4d08-adaa-90541014374b added to full_content_vector.
Embedding for report 8a471315-edd6-461e-aba4-893083caa613 added to full_content_vector.
Embedding for report ac61915a-91f7-4ca1-85ca-286aefdf3a82 added to full_content_vector.
Embedding for report 2ca9b5a6-db